### Merge and clean scraped articles from News API

In [22]:
import os
import glob
import numpy as np
import pandas as pd

In [2]:
keywords = ['merger', 'merge', 'merging', 'merged', 'acquisition', 'acquire', 'acquiring', 'acquired']

In [3]:
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [4]:
all_df = []
for csv_file in csv_files:
    try:
        all_df.append(pd.read_csv(csv_file))
    except:
        print(csv_file)
        all_df.append(pd.read_csv(csv_file, lineterminator='\n'))

/Users/ryanong/Desktop/M&A_KG/acquire_2021-01-01_2021-06-30.csv
/Users/ryanong/Desktop/M&A_KG/acquire_2020-07-01_2020-12-31.csv


In [5]:
master_df = pd.concat(all_df)

In [6]:
# remove duplications
master_df = master_df[~master_df.duplicated(subset=['title'])]
master_df = master_df[~master_df.duplicated(subset=['description'])]
master_df = master_df[~master_df.duplicated(subset=['content'])]

In [7]:
master_df.reset_index(drop=True, inplace = True)

In [8]:
master_df

,source,author,title,description,url,urlToImage,publishedAt,content,keywords
0,"{'id': 'engadget', 'name': 'Engadget'}",Mariella Moon,Lawmakers urge the FCC to seek public input on...,T-Mobile and Sprint found a powerful ally in F...,https://www.engadget.com/2019/08/17/democratic...,https://o.aolcdn.com/images/dims?thumbnail=120...,2019-08-17T07:28:00Z,The DOJ only endorsed the deal after the carri...,merger
1,"{'id': 'the-verge', 'name': 'The Verge'}",Dieter Bohn,T-Mobile’s latest merger gambit isn’t subtle,T-Mobile is trying to manipulate public opinio...,https://www.theverge.com/2019/11/8/20954479/t-...,https://cdn.vox-cdn.com/thumbor/yak18RxzSLehKa...,2019-11-08T12:00:00Z,John Legere trades a pink t-shirt for a firema...,merger
2,"{'id': 'the-verge', 'name': 'The Verge'}",Andrew Marino,Will the Sprint and T-Mobile merger create com...,Verge editor-in-chief Nilay Patel talks to Gig...,https://www.theverge.com/2019/10/22/20924775/p...,https://cdn.vox-cdn.com/thumbor/SSrEMRaiydxtOS...,2019-10-22T17:01:41Z,Interview with tech policy expert Gigi Sohn\r\...,merger
3,"{'id': 'the-verge', 'name': 'The Verge'}",Sean O'Kane,Vizio accuses LeEco of breaching settlement ov...,The television company says LeEco allegedly st...,https://www.theverge.com/2019/12/31/21044504/v...,https://cdn.vox-cdn.com/thumbor/DnNm2wjALdZss0...,2019-12-31T17:28:17Z,LeEco allegedly still owes a substantial amoun...,merger
4,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Sarah Perez,ViacomCBS shakes up its content leadership tea...,Following the merger of CBS and Viacom announc...,http://techcrunch.com/2019/11/11/viacomcbs-sha...,https://techcrunch.com/wp-content/uploads/2019...,2019-11-11T16:32:16Z,Following the merger of CBS and Viacom announc...,merger
...,...,...,...,...,...,...,...,...,...
92901,"{'id': None, 'name': 'WSOC Charlotte'}",EndPlay,Former Walmart in south Charlotte acquired for...,A former Walmart in south Charlotte has been a...,https://www.wsoctv.com/news/local/former-walma...,https://mediaweb.wsoctv.com/photo/2019/07/13/A...,2019-07-13T17:28:55Z,"<ul></ul>\r\nCHARLOTTE, N.C. - A former Walmar...",acquired
92902,"{'id': None, 'name': 'WSOC Charlotte'}",John Downey,Duke Energy tower under construction in uptown...,Maryland-based CGA Capital has partnered with ...,https://www.wsoctv.com/news/local/duke-energy-...,https://www.wsoctv.com/resizer/6Os_duw0pJVuU5E...,2019-12-25T22:17:11Z,"CHARLOTTE, N.C. — Maryland-based CGA Capital h...",acquired
92903,"{'id': None, 'name': 'WCVB Boston'}",NaN,"Largest furniture, flooring store in New Engla...",Rotmans is one of the largest independent furn...,https://www.wcvb.com/article/worcester-based-v...,https://kubrick.htvapps.com/htv-prod-media.s3....,2019-07-22T13:23:00Z,"WORCESTER, Mass. —The largest furniture and fl...",acquired
92904,"{'id': None, 'name': 'Euronews'}",NaN,Cyprus to strip 26 people of citizenship acqui...,The small Mediterranean island has had a citiz...,https://www.euronews.com/2019/11/08/cyprus-to-...,https://static.euronews.com/articles/stories/0...,2019-11-08T09:59:18Z,Cyprus is to strip 26 people of their golden p...,acquired


In [9]:
master_df['keywords'].value_counts()

acquisition    30187
merger         22483
acquire        20910
merge           6757
acquired        6588
acquiring       2761
merging         2038
merged          1182
Name: keywords, dtype: int64

In [22]:
from datetime import datetime
from segtok.segmenter import split_single

In [12]:
master_df['date_obj'] = master_df['publishedAt'].apply(lambda x: datetime.fromisoformat(x[:-1]))

In [13]:
master_df['year'] = master_df['date_obj'].apply(lambda x: x.year)
master_df['month'] = master_df['date_obj'].apply(lambda x: x.month)

In [73]:
master_df.sort_values(by='date_obj', inplace = True)
master_df.reset_index(drop = True, inplace = True)

In [83]:
def extract_key_sentences(content, keyword):
    try:
        sentences = [(sent, keyword) for sent in split_single(content) if keyword in sent]
    except:
        return None
    return sentences

In [84]:
master_df['key_sentences'] = master_df.apply(lambda x: extract_key_sentences(x['content'], x['keywords']), axis = 1)

In [134]:
master_df = master_df[master_df['key_sentences'].str.len() != 0]
master_df.dropna(inplace = True)
master_df.reset_index(drop = False, inplace = True)

In [135]:
master_df['year'].value_counts()

2018    16750
2019    14448
2020    13480
2021     9008
Name: year, dtype: int64

In [137]:
master_df['key_sentences'] = master_df.apply(lambda x: [tuple([x['index']]+list(tup)) for tup in x['key_sentences']], axis = 1)

In [153]:
master_df['keywords'].value_counts()

acquisition    16809
merger         15669
acquire        11655
merge           3881
acquired        3630
acquiring        975
merging          658
merged           409
Name: keywords, dtype: int64

In [139]:
master_df.to_csv('master_data.csv', index = False)

### Breakdown master_data into years & articles and sentences level

In [20]:
import ast
import pandas as pd

In [2]:
master_df = pd.read_csv('master_data.csv')

In [3]:
master_df['key_sentences_array'] = master_df['key_sentences'].apply(lambda x: ast.literal_eval(x))
master_df.rename(columns={'index': 'article_id'}, inplace = True)

In [9]:
master_df['year'].value_counts()

2018    16750
2019    14448
2020    13480
2021     9008
Name: year, dtype: int64

In [4]:
df_2018 = master_df[master_df['year'] == 2018]

flat_list = [item for sublist in list(df_2018['key_sentences_array'].dropna()) for item in sublist]
sentence_2018_df = pd.DataFrame(flat_list, columns=['article_id', 'sentences', 'keywords'])

In [5]:
df_2019 = master_df[master_df['year'] == 2019]

flat_list = [item for sublist in list(df_2019['key_sentences_array'].dropna()) for item in sublist]
sentence_2019_df = pd.DataFrame(flat_list, columns=['article_id', 'sentences', 'keywords'])

In [6]:
df_2020 = master_df[master_df['year'] == 2020]

flat_list = [item for sublist in list(df_2020['key_sentences_array'].dropna()) for item in sublist]
sentence_2020_df = pd.DataFrame(flat_list, columns=['article_id', 'sentences', 'keywords'])

In [7]:
df_2021 = master_df[master_df['year'] == 2021]

flat_list = [item for sublist in list(df_2021['key_sentences_array'].dropna()) for item in sublist]
sentence_2021_df = pd.DataFrame(flat_list, columns=['article_id', 'sentences', 'keywords'])

In [26]:
print('2018 data')
print('---------')
print('No. articles: ' + str(len(df_2018)))
print('No. sentences: ' + str(len(sentence_2018_df)))
print('\n')

print('2019 data')
print('---------')
print('No. articles: ' + str(len(df_2019)))
print('No. sentences: ' + str(len(sentence_2019_df)))
print('\n')

print('2020 data')
print('---------')
print('No. articles: ' + str(len(df_2020)))
print('No. sentences: ' + str(len(sentence_2020_df)))
print('\n')

print('2021 data')
print('---------')
print('No. articles: ' + str(len(df_2021)))
print('No. sentences: ' + str(len(sentence_2021_df)))

2018 data
---------
No. articles: 16750
No. sentences: 47704


2019 data
---------
No. articles: 14448
No. sentences: 44635


2020 data
---------
No. articles: 13480
No. sentences: 39379


2021 data
---------
No. articles: 9008
No. sentences: 23995


In [29]:
df_2018.to_csv('2018_raw_articles_data.csv', index = False)
sentence_2018_df.to_csv('2018_raw_sentences_data.csv', index = False)

df_2019.to_csv('2019_raw_articles_data.csv', index = False)
sentence_2019_df.to_csv('2019_raw_sentences_data.csv', index = False)

df_2020.to_csv('2020_raw_articles_data.csv', index = False)
sentence_2020_df.to_csv('2020_raw_sentences_data.csv', index = False)

df_2021.to_csv('2021_raw_articles_data.csv', index = False)
sentence_2021_df.to_csv('2021_raw_sentences_data.csv', index = False)

### Create JSON files using 2018 articles for NER Training

Preparation of 3 batches of data for three annotators

47704 is the total with 3 annotators

1. Take out 1600 common samples with close to equal sample of each keyword
2. Split the remaining into 3 boxes
3. Add the 1600 to the split boxes

We will end up with three batches of 16968 data samples for each annotator

In [1]:
import json

In [11]:
batch_of_two_hundreds = []
for key in sentence_2018_df['keywords'].value_counts().index:
    batch_of_two_hundreds.append(sentence_2018_df[sentence_2018_df['keywords'] == key][:200])

In [12]:
batch_of_two_hundreds_df = pd.concat(batch_of_two_hundreds)

In [13]:
batch_of_two_hundreds_df['keywords'].value_counts()

acquire        200
merge          200
acquisition    200
merging        200
acquiring      200
acquired       200
merged         200
merger         200
Name: keywords, dtype: int64

In [14]:
the_rest = []
for key in sentence_2018_df['keywords'].value_counts().index:
    the_rest.append(sentence_2018_df[sentence_2018_df['keywords'] == key][200:])

In [15]:
the_rest_df = pd.concat(the_rest)

In [16]:
the_rest_df['keywords'].value_counts()

merger         19819
acquisition    14856
acquire         5644
merge           3731
acquired        1724
acquiring        222
merging          103
merged             5
Name: keywords, dtype: int64

In [17]:
the_rest_df.reset_index(drop = True, inplace = True)

In [18]:
the_rest_df

,article_id,sentences,keywords
0,261,The potential merger would be “a merger of equ...,merger
1,263,MANILA - The education units of the Yuchengco ...,merger
2,263,Ayala Corp's AC Education Inc and the Yuchengc...,merger
3,263,The merger would result in a combined student ...,merger
4,263,Ayala Corp chairperson Jaime Augusto Zobel de ...,merger
...,...,...,...
46099,16601,The fate of the villages which were merged wit...,merged
46100,16601,The municipal Act also showed them as merged v...,merged
46101,16649,Despite ensuring that I stayed only at the SPG...,merged
46102,16649,My SPG number is XXXXXXXX650 (recently merged ...,merged


In [19]:
shuffled = the_rest_df.sample(frac=1)

In [23]:
result = np.array_split(shuffled, 3)

In [26]:
dataset1 = result[0].append(batch_of_two_hundreds_df)

In [27]:
dataset2 = result[1].append(batch_of_two_hundreds_df)

In [28]:
dataset3 = result[2].append(batch_of_two_hundreds_df)

In [29]:
dataset1.reset_index(drop = True, inplace = True)
dataset2.reset_index(drop = True, inplace = True)
dataset3.reset_index(drop = True, inplace = True)

In [31]:
batch_of_two_hundreds_df.to_csv('common_batch.csv', index = False)

In [59]:
with open("dataset1.json", "w") as outfile:
    for i in range(len(dataset1)):
        data = {"text": dataset1['sentences'][i], "label": []}
        json.dump(data, outfile)
        outfile.write('\n')

In [60]:
with open("dataset2.json", "w") as outfile:
    for i in range(len(dataset2)):
        data = {"text": dataset2['sentences'][i], "label": []}
        json.dump(data, outfile)
        outfile.write('\n')

In [61]:
with open("dataset3.json", "w") as outfile:
    for i in range(len(dataset3)):
        data = {"text": dataset3['sentences'][i], "label": []}
        json.dump(data, outfile)
        outfile.write('\n')